In [1]:
import sys
sys.path.append("../")
sys.path.append("../src")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

testing are done in `test.py`

In [2]:
result = pd.read_csv("./test_results.csv")

Interpretation of Columns
- `similarity_response_answer`: cosine similarity between response and manually labeled answer
- `similarity_response_question`: cosine similarity between response and input question

In [3]:
result.head()

,Question,model,parser,similarity_response_question,similarity_response_answer
0,Are investment theses predicated on the contin...,llama2,llamaparse,0.784185,0.958332
1,Are investment theses predicated on the contin...,llama2,pdfplumber,0.877712,0.809557
2,Are investment theses predicated on the contin...,llama2,pypdfloader,0.874767,0.814908
3,Are investment theses predicated on the contin...,nlp_base,llamaparse,0.728865,0.784294
4,Are investment theses predicated on the contin...,nlp_base,pdfplumber,0.742610,0.772052


In [4]:
result.iloc[:, 1:].groupby(["model", "parser"]).mean()

similarity_response_question  \
model         parser                                      
llama2        llamaparse                       0.845017   
              pdfplumber                       0.852351   
              pypdfloader                      0.857827   
nlp_base      llamaparse                       0.745874   
              pdfplumber                       0.759380   
              pypdfloader                      0.741713   
nlp_langchain llamaparse                       0.848223   
              pdfplumber                       0.759380   
              pypdfloader                      0.741713   

                           similarity_response_answer  
model         parser                                   
llama2        llamaparse                     0.832851  
              pdfplumber                     0.816719  
              pypdfloader                    0.824227  
nlp_base      llamaparse                     0.791079  
              pdfplumber                     0.806773  
              pypdfloader                    0.792598  
nlp_langchain llamaparse                     0.837659  
              pdfplumber                     0.776838  
              pypdfloader                    0.788849